In [1]:
# Importing Labraries
import tkinter as tk
from tkinter import filedialog
import pandas as pd
from openpyxl import load_workbook
import xlrd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from datetime import datetime, timedelta
import re
import os
import numpy as np
import sys
from PySide6.QtWidgets import (
    QApplication, QWidget, QLabel, QPushButton, QVBoxLayout,
    QHBoxLayout, QGroupBox, QMessageBox, QRadioButton, QButtonGroup
)
from PySide6.QtGui import QPixmap
from PySide6.QtCore import Qt
import sys


In [2]:
# classe planilhas
class planilha:
    def get_address(self):
        # Cria uma janela oculta
        root = tk.Tk()
        root.withdraw()
        # Abre a janela para selecionar o arquivo
        try:
            caminho_arquivo = filedialog.askopenfilename(
                title="Selecione um arquivo",
                filetypes=[("Arquivos Excel", "*.xls"),("Arquivos Excel", "*.xlsx"), ("Todos os arquivos", "*.*")]
            )
            return caminho_arquivo
        except Exception as error:
            print(f"error type:{error}")
            
    def get_new_address(self,address): 
        address_splited=address.split("/")
        old_name_splited=address_splited[-1].split(".")
        new_name=old_name_splited[0]+"_MODIFIED_SPREADSHEET.xlsx"
        new_address = "/".join(address_splited[:-1]) + "/" + new_name
        return new_address

    def copy_spreadsheet(self,old_address,new_address):
        with open(old_address, 'rb') as origem:
            conteudo = origem.read()
        try:
            with open(new_address, 'wb') as destino:
                destino.write(conteudo)
            print("Arquivo Excel copiado com sucesso!")
        except Exception as error:
            print(f"error type:{error}")
   
    def flow(self):
        g_address=self.get_address()
        n_address=self.get_new_address(g_address)
        self.copy_spreadsheet(g_address,n_address)
        return n_address

In [3]:
# Classe Spreadsheet
class SpreadsheetMetadata:
    def __init__(self,address):
        self.address = address
        self.cabecalho=self.get_cabecalho(self.address)
        self.cia=self.set_cia()
        self.start_billing_period=self.set_start_billing_period()
        self.end_billing_period=self.set_end_billing_period()
        
    def get_cabecalho(self, num_linhas=6):
        ws = load_workbook(filename=self.address, data_only=True).active
        cabecalho=[row for row in ws.iter_rows(min_row=1, max_row=6, values_only=True)]
        return cabecalho

    def set_cia(self):
        return self.cabecalho[1][2]
        
    def set_start_billing_period(self):
        date=re.split(r"[ /]",self.cabecalho[1][5])
        start_str="-".join([date[0],date[1],date[-1]])+" 00:00:00"        
        start = datetime.strptime(start_str, "%d-%m-%Y %H:%M:%S")        
        return start
        
    def set_end_billing_period(self):
        date=re.split(r"[ /]",self.cabecalho[1][5])
        end_str="-".join([date[-3],date[-2],date[-1]])+" 23:59:59"      
        end=datetime.strptime(end_str, "%d-%m-%Y %H:%M:%S")+timedelta(seconds=1)      
        return end   


In [5]:
# Classe Cia Aérea
class CiaAerea:
    def __init__(self,address):
        self.address=address
        self.metadata=SpreadsheetMetadata(address)
        self.table=self.get_table()  
        
    def get_sheet_name(self):
        return None
        
    def get_table(self):
        name = self.get_sheet_name()
        tb=pd.read_excel(self.address,sheet_name=name,skiprows=6)
        return tb

    def _loc_teste(self,name_df,columns,ty):
        df = getattr(self, name_df)
        for col in columns:
            df[col] = df[col].astype(ty)
        setattr(self,name_df,df)


    def data_structure(self):
        keys_cols={"string":["Localizador",
                             "Bilhete", "Órgão Solicitante",
                             "Órgão Superior",
                             "PCDP" ,
                             "Evento(s) de Faturamento",
                             "Situação do Bilhete",
                             "Situações de Embarque",
                             "Status do Faturamento",
                             "Tipo(s) de Pendência",
                             "Análise da Fiscalização",
                             "Análise da Companhia Aérea"],
                  "datetime64[ns]":["Data de emissão",
                                    "Data de Cancelamento"],
                  "float64":['Crédito de Reembolso',
                             'Desconto %',
                             'Desconto Efetivo %',
                             'Desconto Efetivo R$',
                             'Desconto R$',
                             'Glosa',
                             'Multa Calculada',
                             'Multa Efetiva',
                             'Multa de Reembolso Retornada',
                             'Reembolso Calculado',
                             'Reembolso Efetivo',
                             'Reembolso Retornado',
                             'Tarifa Comercial Retornada',
                             'Tarifa Embarque Efetiva',
                             'Tarifa Praticada Calculada',
                             'Tarifa Praticada Efetiva',
                             'Tarifa Praticada Retornada',
                             'Tarifa de Embarque Retornada',
                             'Total',
                             'Valor a Faturar']
                  }
        for key in keys_cols.keys():
            self._loc_teste("table",keys_cols[key],key)

    def insert_message_on_tickets_with_discount_bellow_three_percent(self):
        df = getattr(self, "table") 
        filter_discount_below_three_percent= df[df["Tipo(s) de Pendência"].str.contains("Pendente de Desconto", na=False)]
        new_text="Validar o valor a faturar - Não foi aplicado o desconto - bilhete glosado"
        df.loc[filter_discount_below_three_percent.index,"Análise da Fiscalização"]=(df.loc[filter_discount_below_three_percent.index,"Análise da Fiscalização"]
                                                                                      .apply(lambda x: new_text if pd.isna(x) else f"{x}\n{new_text}"
                                                                                                                                                                        )
                                                                                     )
        setattr(self,"table",df)
    
      
    def general_message(self):
        df = getattr(self, "table") 
        df.loc[df["Análise da Fiscalização"].isna(),"Análise da Fiscalização"]="Validar o valor a faturar"
        setattr(self,"table",df)


    def change_worksheet(self):
        wb = load_workbook(self.address)
        ws = wb[self.get_sheet_name()] 
        for i, valor in enumerate(self.table["Análise da Fiscalização"], start=8):
            ws.cell(row=i, column=33, value=valor)
        wb.save(self.address)   


In [9]:
# Classe Cia Aérea Emitidos
class CiaAreaIssued(CiaAerea):
    def __init__(self,address):
        super().__init__(address)
        self.table=self.get_table()   

    def get_sheet_name(self):
        return "Emitidos"

    def insert_message_on_tickets_canceled_after_the_billing_period(self):
        df = getattr(self, "table") 
        #tbl=self.tickets_issued.copy()
        filter_billing_period=df[(~df['Data de Cancelamento'].isnull()
                                )&(df['Data de Cancelamento']>=self.metadata.end_billing_period)
                                ]
        new_text="Cancelado no mês seguinte ao de referência. Será reembolsado no próximo faturamento."
        
        df.loc[filter_billing_period.index, "Análise da Fiscalização"] = (df.loc[filter_billing_period.index, "Análise da Fiscalização"]
                                                                           .apply(lambda x: new_text if pd.isna(x) else f"{x}\n{new_text}"
                                                                                 )
                                                                          )
        setattr(self,"table",df)

    def insert_message_about_72_hours(self):
        df = getattr(self, "table") 
        filter_message_about_72_hours= df[df["Tipo(s) de Pendência"].str.contains("Pendende de Reserva", na=False)]
        
        new_text="Validar o valor a faturar - Não foi respeitado o prazo de 72 horas da reserva"
        df.loc[filter_message_about_72_hours.index,"Análise da Fiscalização"]=(df.loc[filter_message_about_72_hours.index,"Análise da Fiscalização"]
                                                                                      .apply(lambda x: new_text if pd.isna(x) else f"{x}\n{new_text}"
                                                                                                                                                                        )
                                                                                     )
        setattr(self,"table",df)
        
    def flow(self):
        self.data_structure()
        self.insert_message_on_tickets_canceled_after_the_billing_period()
        self.insert_message_on_tickets_with_discount_bellow_three_percent()
        self.insert_message_about_72_hours()
        self.general_message()
        self.change_worksheet()


In [10]:
# Classe emitidos e Cancelados  
class CiaAreaIssuedCancelled(CiaAerea):
    def __init__(self,address):
        super().__init__(address)
        self.table=self.get_table()   

    def get_sheet_name(self):
        return "Emitidos e Cancelados"
        
    def void_tickets(self):
        df = getattr(self, "table") 
        df.loc[df["Status do Faturamento"] == "Não Faturável", "Análise da Fiscalização"] = "VOID"
        setattr(self,"table",df)

    def insert_message_on_tickets_with_discount_bellow_three_percent(self):
        df = getattr(self, "table") 
        filter_discount_below_three_percent=df[df['Desconto %']<0.03]
        new_text="Validar o valor do reembolso indicado na coluna AA - Não foi aplicado o desconto - bilhete glosado"
        df.loc[filter_discount_below_three_percent.index,"Análise da Fiscalização"]=(df.loc[filter_discount_below_three_percent.index,"Análise da Fiscalização"]
                                                                                      .apply(lambda x: new_text if pd.isna(x) else f"{x}\n{new_text}"
                                                                                            )
                                                                                    )

    def general_message(self):
        df = getattr(self, "table") 
        df.loc[df["Análise da Fiscalização"].isna(),"Análise da Fiscalização"]="Validar o valor do reembolso indicado na coluna AA"
        setattr(self,"table",df)    
    
    
    def flow(self):
        self.data_structure()
        self.void_tickets()
        self.insert_message_on_tickets_with_discount_bellow_three_percent()
        self.general_message()
        self.change_worksheet()

# TESTE
<HR></HR>

In [11]:
worksheet_excel = planilha()
address = worksheet_excel.flow()
issued = CiaAreaIssued(address)
issued.flow()

Arquivo Excel copiado com sucesso!


C:\Users\JuniorZancanaro\.conda\envs\Aeropy\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\JuniorZancanaro\.conda\envs\Aeropy\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\JuniorZancanaro\.conda\envs\Aeropy\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [12]:
ic=CiaAreaIssuedCancelled(address)


In [19]:
ic.table.loc[1,"Reembolso Retornado"]

np.float64(1478.88)

In [16]:
ic.table[ic.table['Reembolso Retornado']>ic.table['Reembolso Efetivo']]

,Localizador,Bilhete,Órgão Solicitante,Órgão Superior,PCDP,Data de emissão,Evento(s) de Faturamento,Data de Cancelamento,Situação do Bilhete,Situações de Embarque,...,Multa Efetiva,Reembolso Efetivo,Total,Crédito de Reembolso,Valor a Faturar,Status do Faturamento,Tipo(s) de Pendência,Análise da Fiscalização,Relatório de Faturamento Definitivo da Emissão,Análise da Companhia Aérea
1,HFKLNY,HFKLNY,Comitê Intersetorial de Acompanhamento e Monit...,Ministério dos Direitos Humanos e da Cidadania,001657/25,2025-07-20,Emissão / Cancelamento,2025-07-20,Cancelado,Indefinido / Indefinido,...,0.00,0.00,0.00,0,0.00,Não Faturável,NaN,NaN,NaN,NaN
3,OE9ZRB,OE9ZRB,Comitê Intersetorial de Acompanhamento e Monit...,Ministério dos Direitos Humanos e da Cidadania,001656/25,2025-07-19,Emissão / Cancelamento,2025-07-19,Cancelado,Indefinido / Indefinido,...,0.00,0.00,0.00,0,0.00,Não Faturável,NaN,NaN,NaN,NaN
4,WQWI5H,WQWI5H,Comitê Intersetorial de Acompanhamento e Monit...,Ministério dos Direitos Humanos e da Cidadania,001655/25,2025-07-19,Emissão / Cancelamento,2025-07-19,Cancelado,Indefinido / Indefinido,...,0.00,0.00,0.00,0,0.00,Não Faturável,NaN,NaN,NaN,NaN
5,YWGIPQ,YWGIPQ,Comitê Intersetorial de Acompanhamento e Monit...,Ministério dos Direitos Humanos e da Cidadania,001656/25,2025-07-19,Emissão / Cancelamento,2025-07-19,Cancelado,Indefinido / Indefinido,...,0.00,0.00,0.00,0,0.00,Não Faturável,NaN,NaN,NaN,NaN
6,UH5PKV,UH5PKV,Comitê Intersetorial de Acompanhamento e Monit...,Ministério dos Direitos Humanos e da Cidadania,001655/25,2025-07-19,Emissão / Cancelamento,2025-07-19,Cancelado,Indefinido / Indefinido,...,0.00,0.00,0.00,0,0.00,Não Faturável,NaN,NaN,NaN,NaN
7,EW66YN,EW66YN,Comitê Intersetorial de Acompanhamento e Monit...,Ministério dos Direitos Humanos e da Cidadania,001653/25,2025-07-19,Emissão / Cancelamento,2025-07-20,Cancelado,Indefinido / Indefinido,...,1307.01,60.62,1224.30,0,1224.30,Pendente,Pendente de Reembolso,NaN,NaN,NaN
8,YQKITK,YQKITK,Comitê Intersetorial de Acompanhamento e Monit...,Ministério dos Direitos Humanos e da Cidadania,001642/25,2025-07-19,Emissão / Cancelamento,2025-07-19,Cancelado,Indefinido / Indefinido,...,0.00,0.00,0.00,0,0.00,Não Faturável,NaN,NaN,NaN,NaN
9,DIDL3M,DIDL3M,Comitê Intersetorial de Acompanhamento e Monit...,Ministério dos Direitos Humanos e da Cidadania,001647/25,2025-07-19,Emissão / Cancelamento,2025-07-19,Cancelado,Indefinido / Indefinido,...,0.00,0.00,0.00,0,0.00,Não Faturável,NaN,NaN,NaN,NaN
10,SH7WPV,SH7WPV,Comitê Intersetorial de Acompanhamento e Monit...,Ministério dos Direitos Humanos e da Cidadania,001642/25,2025-07-19,Emissão / Cancelamento,2025-07-19,Cancelado,Indefinido / Indefinido,...,0.00,0.00,0.00,0,0.00,Não Faturável,NaN,NaN,NaN,NaN
12,NGLIMI,NGLIMI,Comitê Intersetorial de Acompanhamento e Monit...,Ministério dos Direitos Humanos e da Cidadania,001647/25,2025-07-19,Emissão / Cancelamento,2025-07-19,Cancelado,Indefinido / Indefinido,...,0.00,0.00,0.00,0,0.00,Não Faturável,NaN,NaN,NaN,NaN


In [24]:
issued.__dict__.keys()

dict_keys(['address', 'metadata', 'table'])

In [11]:
dir(issued)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_loc_teste',
 'address',
 'change_worksheet',
 'data_structure',
 'flow',
 'general_message',
 'get_sheet_name',
 'get_table',
 'insert_message_about_72_hours',
 'insert_message_on_tickets_canceled_after_the_billing_period',
 'insert_message_on_tickets_with_discount_bellow_three_percent',
 'metadata',
 'table']

In [25]:
issued.data_structure()
issued.insert_message_on_tickets_canceled_after_the_billing_period()


In [47]:
a=130
issued.table.loc[146]

Localizador                                                                EIIWNU
Bilhete                                                             9572233515333
Órgão Solicitante                     Secretaria Nacional de Saneamento Ambiental
Órgão Superior                                             Ministério das Cidades
PCDP                                                                    000562/25
Data de emissão                                               2025-05-22 00:00:00
Evento(s) de Faturamento                                                  Emissão
Data de Cancelamento                                          2025-05-15 00:00:00
Situação do Bilhete                                                       Emitido
Situações de Embarque                                       Utilizado / Utilizado
Tarifa Praticada Retornada                                                 3499.0
Tarifa de Embarque Retornada                                                36.67
Tarifa Comercial

In [34]:
issued.table["Análise da Fiscalização"].iloc[4]

'Validar o valor a faturar - Não foi respeitado o prazo de 72 horas da reserva'

In [ ]:
        
    def flow(self):
        self.data_structure()
        self.insert_message_on_tickets_canceled_after_the_billing_period()
        self.insert_message_on_tickets_with_discount_bellow_three_percent()
        self.insert_message_about_72_hours()
        self.general_message()
        self.change_worksheet()